In [ ]:
conda activate openfold-env

In [2]:
import os
import re
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint

In [32]:
%load_ext autoreload
%autoreload 2 

project_dir = os.path.dirname(os.getcwd())
print('project direcotry is ', project_dir)
if project_dir not in sys.path:
    sys.path.append(project_dir)
print(sys.path)

#
pdb_dir = '/home/yuan/data/pdb/rsync'
data_dir = './data/'
outputs_dir = '/home/yuan/output/pdb_abag'

from src.db_build import DbBuild
from src.db_query import DbQuery
from src.db_update import DbUpdate

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
project direcotry is  /home/yuan/bio
['/home/yuan/anaconda3/envs/openfold-env/lib/python310.zip', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/lib-dynload', '', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages/openfold-2.2.0-py3.10-linux-x86_64.egg', '/home/yuan/bio']


## 1. label data for abag

### 1.1 by heavy/Light

In [252]:
query = """
    select A.chain, A.compound_no, A.pdb_id, A.chain_type, A.fragment, A.compound_text, B.name, B.pdb_url
    from compound A, pdb B
    where A.pdb_id = B.pdb_id 
        AND A.pdb_id IN (
            select distinct pdb_id
            from compound
            where chain_type REGEXP "heavy|light"
        )
    order by A.pdb_id, compound_no;
"""
df = DbQuery().list_data(query, True)
outfile = os.path.join(data_dir, 'abag_HL.xlsx')
df.to_excel(outfile, index=False)

(14053, 8)


In [266]:
# verify again
infile = os.path.join(data_dir, 'abag_HL.xlsx')
xls_file = pd.ExcelFile(infile)
df = xls_file.parse('Sheet1')
for name, sub in df.groupby('pdb_id'):
    chain_types = list(sub['chain_type'])
    if 'heavy' not in chain_types and 'light' not in chain_types and 'single' not in chain_types:
        print(name, chain_types)
    elif 'x' in chain_types:
        print(name, chain_types)

In [272]:
# update compount.chain_type
infile = os.path.join(data_dir, 'label', 'abag_HL.xlsx')
xls_file = pd.ExcelFile(infile)
table = xls_file.parse('Sheet1')
table.head()

,chain,compound_no,pdb_id,chain_type,fragment,compound_text,name,pdb_url
0,"l,m",1,12E8,light,fab,molecule: igg1-kappa 2e8 fab (light chain) ; f...,2e8 fab fragment,https://www.rcsb.org/structure/12E8
1,"h,p",2,12E8,heavy,fab,molecule: igg1-kappa 2e8 fab (heavy chain) ; f...,2e8 fab fragment,https://www.rcsb.org/structure/12E8
2,l,1,15C8,light,fab,molecule: igg 5c8 fab (light chain) ; fragment...,"catalytic antibody 5c8, free fab",https://www.rcsb.org/structure/15C8
3,h,2,15C8,heavy,fab,molecule: igg 5c8 fab (heavy chain) ; fragment...,"catalytic antibody 5c8, free fab",https://www.rcsb.org/structure/15C8
4,l,1,1A0Q,light,fab,molecule: 29g11 fab (light chain) ; fragment: ...,29g11 complexed with phenyl [1-(1-n-succinylam...,https://www.rcsb.org/structure/1A0Q


In [299]:
# insert data into table abag
from src.db_build import DbBuild
DbBuild(table).insert_abag()

table abag: succeed=21148, unchanged=1


In [300]:
# update data
DbQuery().update_compound_table('abag', 'chain_type', table)
DbQuery().update_compound_table('abag', 'fragment', table)

abag.chain_type: succeed=0, unchanged=21149
abag.fragment: succeed=0, unchanged=21149


In [303]:
# update table complex
table['complex_type'] = 'abag'
DbBuild(table).insert_complex()

(6818, 2)
table complex: succeed=0, unchanged=6818


### 1.2 by single

In [306]:
query = """
    select A.chain, A.compound_no, A.pdb_id, A.chain_type, A.fragment, A.compound_text, B.pdb_url
    from compound A, pdb B
    where A.pdb_id = B.pdb_id 
        AND A.pdb_id IN (
            select distinct pdb_id
            from compound
            where chain_type = "single"
        )
    order by A.pdb_id, compound_no;
"""
df = DbQuery().list_data(query, True)
print(df.shape)
outfile = os.path.join(data_dir, 'abag_single.xlsx')
df.to_excel(outfile, index=False)

(8695, 8)


In [317]:
# verify again
infile = os.path.join(data_dir, 'abag_single.xlsx')
xls_file = pd.ExcelFile(infile)
df = xls_file.parse('Sheet1')
for name, sub in df.groupby('pdb_id'):
    chain_types = list(sub['chain_type'])
    if ('heavy' not in chain_types) and ('light' not in chain_types) and ('single' not in chain_types):
        print(name, chain_types)
    elif 'x' in chain_types:
        print(name, chain_types)

In [318]:
# update compount.chain_type
infile = os.path.join(data_dir, 'label', 'abag_single.xlsx')
xls_file = pd.ExcelFile(infile)
table = xls_file.parse('Sheet1')

# insert data into table abag
from src.db_build import DbBuild
DbBuild(table).insert_abag()

# update table abag
DbQuery().update_compound_table('abag', 'chain_type', table)
DbQuery().update_compound_table('abag', 'fragment', table)

table abag: succeed=8162, unchanged=466
abag.chain_type: succeed=1, unchanged=8627
abag.fragment: succeed=3, unchanged=8625
table complex: succeed=2419, unchanged=109


In [ ]:
# insert into table complex
table['complex_type'] = 'abag'
DbBuild(table).insert_complex()

### 1.3 cross validate labelling

In [53]:
from src.db_query import DbQuery

query = """
    select A.chain, A.compound_no, A.pdb_id, A.chain_type, A.fragment, A.compound_text, B.name, B.pdb_url
    from compound A, pdb B
    where A.pdb_id = B.pdb_id 
        AND B.pdb_id IN (
            select distinct pdb_id from compound
            where compound_text LIKE "%fragment%"
                AND compound_text REGEXP "fab|vh|fc|fv"
            except
            select distinct pdb_id from abag
        )
    order by A.pdb_id, A.compound_no;
"""
df = DbQuery().list_data(query, True)
print(df.shape)

outfile = os.path.join(data_dir, 'fragment_fab.xlsx')
df.to_excel(outfile, index=False)

(662, 8)


In [54]:
# update compount.chain_type
infile = os.path.join(data_dir, 'label', 'fragment_fab.xlsx')
xls_file = pd.ExcelFile(infile)
table = xls_file.parse('Sheet1')

# insert data into table abag
DbBuild(table).insert_abag()

# insert into table complex
table['complex_type'] = 'abag'
DbBuild(table).insert_complex()

table abag: total=134, succeed=134, unchanged=0
table complex: total=73, succeed=73, unchanged=0


In [56]:
query = """
    select A.chain, A.compound_no, A.pdb_id, A.chain_type, A.fragment, A.compound_text, B.name, B.pdb_url
    from compound A, pdb B
    where A.pdb_id = B.pdb_id 
        AND B.pdb_id IN (
            select distinct pdb_id from compound
            where B.name REGEXP "[anti|dia|sy|intro|intra|mega|repe|nano|nono|promacro]body|nanobodies|nanoboy"
            except
            select distinct pdb_id from abag
        )
    order by A.pdb_id, A.compound_no;
"""
df = DbQuery().list_data(query, True)
df['fragment'] = 'fv'
print(df.shape)

outfile = os.path.join(data_dir, 'fragment_-body.xlsx')
df.to_excel(outfile, index=False)

(735, 8)


In [57]:
# update compount.chain_type
infile = os.path.join(data_dir, 'label', 'fragment_-body.xlsx')
xls_file = pd.ExcelFile(infile)
table = xls_file.parse('Sheet1')

# insert data into table abag
DbBuild(table).insert_abag()

# insert into table complex
table['complex_type'] = 'abag'
DbBuild(table).insert_complex()

table abag: total=504, succeed=504, unchanged=0
table complex: total=228, succeed=228, unchanged=0


In [60]:
query = """
    select A.chain, A.compound_no, A.pdb_id, A.chain_type, A.fragment, A.compound_text, B.name, B.pdb_url
    from compound A, pdb B
    where A.pdb_id = B.pdb_id 
        AND B.pdb_id IN (
            select distinct pdb_id from compound
            where compound_text REGEXP "igg |igh |immuno"
            except
            select distinct pdb_id from abag
        )
    order by A.pdb_id, A.compound_no;
"""
df = DbQuery().list_data(query, True)
print(df.shape)

outfile = os.path.join(data_dir, 'fragment_igg.xlsx')
df.to_excel(outfile, index=False)

(2826, 8)


In [61]:
# update compount.chain_type
infile = os.path.join(data_dir, 'label', 'fragment_igg.xlsx')
xls_file = pd.ExcelFile(infile)
table = xls_file.parse('Sheet1')

# insert data into table abag
DbBuild(table).insert_abag()

# insert into table complex
table['complex_type'] = 'abag'
DbBuild(table).insert_complex()

table abag: total=33, succeed=33, unchanged=0
table complex: total=17, succeed=17, unchanged=0


## 3. Compared with other database

### 3.1 compared with AACDB

In [76]:
# retrieve abag: heavy/light chains
# create table diff as (select distinct pdb_id from aacdb except select distinct pdb_id from abag);
query = """
    select A.chain, A.compound_no, A.pdb_id, A.chain_type, A.fragment, A.compound_text, B.name, B.pdb_url
    from compound A, pdb B
    where A.pdb_id IN (
            SELECT pdb_id from diff
        ) AND A.pdb_id = B.pdb_id
    order by A.pdb_id, A.compound_no;
"""
df = DbQuery().list_data(query, True)
print(df.shape)

outfile = os.path.join(data_dir, 'diff_aacdb.xlsx')
df.to_excel(outfile, index=False)

(90, 8)


In [77]:
# update compount.chain_type
infile = os.path.join(data_dir, 'label', 'diff_aacdb.xlsx')
xls_file = pd.ExcelFile(infile)
table = xls_file.parse('Sheet1')

# insert data into table abag
DbBuild(table).insert_abag()

# insert into table complex
table['complex_type'] = 'abag'
DbBuild(table).insert_complex()

table abag: total=77, succeed=77, unchanged=0
table complex: total=28, succeed=28, unchanged=0


### 3.2 compared with IEDB

In [79]:
# retrieve abag: heavy/light chains
# create table diff as (select distinct pdb_id from iedb except select distinct pdb_id from abag);
query = """
    select A.chain, A.compound_no, A.pdb_id, A.chain_type, A.fragment, A.compound_text, B.name, B.pdb_url
    from compound A, pdb B
    where A.pdb_id IN (
            SELECT pdb_id from diff
        ) AND A.pdb_id = B.pdb_id
    order by A.pdb_id, A.compound_no;
"""
df = DbQuery().list_data(query, True)
df['chain_type'] = None
df['fragment'] = None
print(df.shape)

outfile = os.path.join(data_dir, 'diff_iedb.xlsx')
df.to_excel(outfile, index=False)

(5441, 8)


In [80]:
# update compount.chain_type
infile = os.path.join(data_dir, 'label', 'diff_iedb.xlsx')
xls_file = pd.ExcelFile(infile)
table = xls_file.parse('Sheet1')

# insert data into table abag
DbBuild(table).insert_abag()

# insert into table complex
table['complex_type'] = 'abag'
DbBuild(table).insert_complex()

table abag: total=124, succeed=124, unchanged=0
table complex: total=38, succeed=38, unchanged=0


### 3.3 compared with SAbDab

In [81]:
# retrieve abag: heavy/light chains
# create table diff as (select distinct pdb_id from sabdab except select distinct pdb_id from abag);
query = """
    select A.chain, A.compound_no, A.pdb_id, A.chain_type, A.fragment, A.compound_text, B.name, B.pdb_url
    from compound A, pdb B
    where A.pdb_id IN (
            SELECT pdb_id from diff
        ) AND A.pdb_id = B.pdb_id
    order by A.pdb_id, A.compound_no;
"""
df = DbQuery().list_data(query, True)
df['chain_type'] = None
df['fragment'] = None
print(df.shape)

outfile = os.path.join(data_dir, 'diff_sabdab.xlsx')
df.to_excel(outfile, index=False)

(345, 8)


In [82]:
# update compount.chain_type
infile = os.path.join(data_dir, 'label', 'diff_sabdab.xlsx')
xls_file = pd.ExcelFile(infile)
table = xls_file.parse('Sheet1')

# insert data into table abag
DbBuild(table).insert_abag()

# insert into table complex
table['complex_type'] = 'abag'
DbBuild(table).insert_complex()

table abag: total=213, succeed=213, unchanged=0
table complex: total=85, succeed=85, unchanged=0


### 3.4 confirm difference

In [334]:
query = """
    select A.chain, C.compound_no, C.pdb_id, C.chain_type, C.fragment, A.compound_text, B.pdb_url
    from compound A, pdb B, abag C
    where A.pdb_id = B.pdb_id
        AND B.pdb_id = C.pdb_id
        AND A.compound_no = C.compound_no
        AND A.pdb_id IN (
            select pdb_id from abag_diff
        )
    order by A.pdb_id, A.compound_no;
"""
df = DbQuery().list_data(query, True)
# print(df.head(10))
print(df.shape)
outfile = os.path.join(data_dir, 'abag_diff.xlsx')
df.to_excel(outfile, index=False)

(520, 7)


In [335]:
# update compount.chain_type
infile = os.path.join(data_dir, 'label', 'abag_diff.xlsx')
xls_file = pd.ExcelFile(infile)
table = xls_file.parse('Sheet1')

# update table abag
DbQuery().update_compound_table('abag', 'chain_type', table)
DbQuery().update_compound_table('abag', 'fragment', table)

abag.chain_type: succeed=56, unchanged=463
abag.fragment: succeed=2, unchanged=517


## 4. update

In [338]:

query = """
    select A.chain, A.compound_no, A.pdb_id, A.chain_type, A.fragment, A.compound_text, B.name, B.pdb_url
    from compound A, pdb B
    where A.pdb_id = B.pdb_id 
        AND B.release_date > '2025-05-16'
    order by A.pdb_id, compound_no;
"""
df = DbQuery().list_data(query, True)
print(df.shape)

def fn(x):
    if re.findall(r'antibody|nanobody', x):
        if re.findall(r'heavy', x):
            return 'heavy'
        elif re.findall(r'light', x):
            return 'light'
        return 'antigen'
df['chain_type'] = df['compound_text'].map(fn)

outfile = os.path.join(data_dir, 'abag_update.xlsx')
df.to_excel(outfile, index=False)

(877, 8)


In [31]:
from src.db_build import DbBuild
from src.db_query import DbQuery

# update compount.chain_type
infile = os.path.join(data_dir, 'label', 'abag_update.xlsx')
xls_file = pd.ExcelFile(infile)
table = xls_file.parse('Sheet1')

# insert into table complex and abag
table['complex_type'] = 'abag'
b = DbBuild(table, verbose=False)
b.insert_complex()
b.insert_abag()

table complex: total=48, succeed=0, unchanged=48
table abag: total=151, succeed=0, unchanged=151


In [83]:
DbQuery().count_complex()

,complex_type,pdb_ids
0,abag,9725
1,ferritin,686
2,kinase,11800
3,MHC,1513
4,myosin,551
5,thrombin,773
